In [ ]:
import os
import getpass


env_content = getpass.getpass()

# 2. This writes the secrets to the Google Server's temporary memory
with open('.env', 'w') as f:
    f.write(env_content)



In [42]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)


In [43]:
print(api_key[:3])

AIz


In [20]:
%pip install chromadb

In [21]:
%pip install sentence-transformers

In [49]:
%pip install langchain-text-splitters

In [52]:
import os
import shutil
import re
import chromadb
from chromadb.utils import embedding_functions
from langchain_text_splitters import MarkdownHeaderTextSplitter


if os.path.exists("db_new"):
    shutil.rmtree("db_new")

# --- 2. SETUP DATABASE (New Folder Name) ---
print("🔌 Setting up new database in 'db_new'...")
local_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# CHANGED PATH HERE from "db/" to "db_new/"
chroma_client = chromadb.PersistentClient(path="db_new/")

🔌 Setting up new database in 'db_new'...


In [53]:
collection = chroma_client.create_collection(
    name="mkdocs_rag",
    embedding_function=local_ef
)

In [46]:
%pip install langchain

In [24]:
%pip install langchain_community

In [25]:
%pip install langchain-text-splitters

In [54]:
# --- 3. DOWNLOAD & PROCESS DATA ---
print("📥 Checking for MkDocs repository...")
if not os.path.exists("mkdocs_repo"):
    !git clone https://github.com/mkdocs/mkdocs.git mkdocs_repo

docs_path = "mkdocs_repo/docs"

headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

print("⏳ Starting ingestion...")
count = 0

for root, dirs, files in os.walk(docs_path):
    for file in files:
        if file.endswith(".md"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read()
            
            cleaned_text = re.sub(r"^---\n(.*?)\n---\n", "", raw_text, flags=re.DOTALL)
            chunks = markdown_splitter.split_text(cleaned_text)
            
            documents = []
            ids = []
            metadatas = []
            
            for i, chunk in enumerate(chunks):
                documents.append(chunk.page_content)
                ids.append(f"{file}-{i}")
                meta = chunk.metadata.copy()
                meta["source"] = file
                metadatas.append(meta)
            
            if documents:
                collection.add(documents=documents, ids=ids, metadatas=metadatas)
                count += 1

print(f"✅ Success! Added {count} files to 'db_new'.")

📥 Checking for MkDocs repository...
Cloning into 'mkdocs_repo'...
remote: Enumerating objects: 17210, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17210 (delta 2), reused 0 (delta 0), pack-reused 17198 (from 3)
Receiving objects: 100% (17210/17210), 32.48 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (11318/11318), done.
⏳ Starting ingestion...
✅ Success! Added 19 files to 'db_new'.


In [55]:
def ask_mkdocs(question):
    print(f"🔎 Searching for: '{question}'...")
    
    # 1. RETRIEVE (Local Search - Fast & Free)
    results = collection.query(
        query_texts=[question],
        n_results=3
    )
    
    # Check if we found any documents
    if not results['documents'][0]:
        print("❌ No relevant documentation found.")
        return

    # Combine the found chunks into one block of text
    retrieved_chunks = results['documents'][0]
    context = "\n\n".join(retrieved_chunks)
    
    # 2. PROMPT (Prepare the instruction for Gemini)
    prompt = f"""
    You are a helpful assistant for the MkDocs open-source project. 
    Use the following pieces of retrieved documentation to answer the user's question.
    If the answer is not in the context, say "I don't know" and do not make up an answer.
    
    CONTEXT:
    {context}
    
    QUESTION:
    {question}
    """
    
    # 3. GENERATE (Send to Google Gemini)
    # We use the 'client' we set up at the very beginning
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents=prompt
        )
        print("-" * 30)
        print(f"📄 Found Context from: {results['metadatas'][0][0]['source']}")
        print("-" * 30)
        print(f"🤖 Answer:\n{response.text}")
        print("=" * 50)
    except Exception as e:
        print(f"❌ Error talking to Google: {e}")



In [56]:
# --- TEST YOUR BOT ---
ask_mkdocs("How do I change the theme of my documentation?")
ask_mkdocs("What is the command to build the static site?")

🔎 Searching for: 'How do I change the theme of my documentation?'...
------------------------------
📄 Found Context from: getting-started.md
------------------------------
🤖 Answer:
To change the theme of your documentation, you need to edit the `mkdocs.yml` file and add a `theme` setting. For example:

```yaml
theme: readthedocs
```

This will change the theme to ReadTheDocs. Save the changes, and you'll see the new theme being used.

🔎 Searching for: 'What is the command to build the static site?'...
------------------------------
📄 Found Context from: getting-started.md
------------------------------
🤖 Answer:
I am sorry, but the provided documentation does not contain the command to build the static site. However, it does say that after building the site, the static files will be located in the `site` directory.

